In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install braindecode==0.4.85
!pip install eeglib

     |████████████████████████████████| 324 kB 4.3 MB/s 
     |████████████████████████████████| 7.4 MB 34.5 MB/s 
  Created wheel for braindecode: filename=Braindecode-0.4.85-py3-none-any.whl size=71094 sha256=f0544cc4eac1570d1cb85f1045e722244c11abecf77684903c74e1ac080a7eb6
  Stored in directory: /root/.cache/pip/wheels/eb/67/4c/81ad85409f913dd7e3ebb4fb5896839680c5627c04d91e3b78
Successfully built braindecode
     |████████████████████████████████| 1.7 MB 5.2 MB/s 


In [3]:
import mne
from braindecode.datasets.bbci import  BBCIDataset
import logging
import sys
import os.path
from collections import OrderedDict
import numpy as np

from braindecode.datasets.bbci import  BBCIDataset
from braindecode.datautil.signalproc import highpass_cnt
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

def load_bbci_data(filename, low_cut_hz):
    load_sensor_names = ['C3', 'C4', 'Cz']
    loader = BBCIDataset(filename, load_sensor_names=load_sensor_names)

    cnt = loader.load()

    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(cnt, marker_def,
                                                  clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = load_sensor_names
    cnt = cnt.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    cnt = resample_cnt(cnt, 250.0)
    cnt = mne_apply(
        lambda a: highpass_cnt(
            a, low_cut_hz, cnt.info['sfreq'], filt_order=3, axis=1),
        cnt)
    cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                  init_block_size=1000,
                                                  eps=1e-4).T,
        cnt)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [0, 2000]

    dataset = create_signal_target_from_raw_mne(cnt, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset

list_a = [] #存放所有.mat文件的路径
dir_str = '/content/drive/MyDrive/EEG-dataset/high-gamma-dataset'
for i in range(1, 29):
  list_a.append(os.path.join(dir_str, "a"+str(i)+".mat"))

datasets = []
for mat_path in list_a:
  datasets.append(load_bbci_data(mat_path, 0))

Creating RawArray with float64 data, n_channels=3, n_times=1225545
    Range : 0 ... 1225544 =      0.000 ...  2451.088 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/datasets/bbci.py:210: DeprecationWarning: events cannot be set directly. This warning will turn into an error after 0.24
  cnt.info["events"] = np.array(event_arr).T
This is not causal, uses future data....


Creating RawArray with float64 data, n_channels=3, n_times=612772
    Range : 0 ... 612771 =      0.000 ...  2451.084 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/mne_ext/signalproc.py:67: DeprecationWarning: sfreq cannot be set directly. Please use method inst.resample() instead. This warning will turn into an error after 0.24
  new_info["sfreq"] = new_fs


Creating RawArray with float64 data, n_channels=3, n_times=3131420
    Range : 0 ... 3131419 =      0.000 ...  6262.838 secs
Ready.


...

/usr/local/lib/python3.7/dist-packages/braindecode/mne_ext/signalproc.py:67: DeprecationWarning: sfreq cannot be set directly. Please use method inst.resample() instead. This warning will turn into an error after 0.24
  new_info["sfreq"] = new_fs


Creating RawArray with float64 data, n_channels=3, n_times=608505
    Range : 0 ... 608504 =      0.000 ...  1217.008 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/datasets/bbci.py:210: DeprecationWarning: events cannot be set directly. This warning will turn into an error after 0.24
  cnt.info["events"] = np.array(event_arr).T
This is not causal, uses future data....


Creating RawArray with float64 data, n_channels=3, n_times=304252
    Range : 0 ... 304251 =      0.000 ...  1217.004 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/mne_ext/signalproc.py:67: DeprecationWarning: sfreq cannot be set directly. Please use method inst.resample() instead. This warning will turn into an error after 0.24
  new_info["sfreq"] = new_fs


In [4]:
import numpy as np
train_X, train_y = [], []
for D in datasets:
  train_X.append(np.transpose(np.asarray(D.X), [0, 2, 1]).reshape([-1, 25, 20, 3]))
  train_y.append(np.asarray(D.y))
train_X = np.concatenate(train_X, axis=0)
train_y = np.concatenate(train_y, axis=0)
train_X.shape

(13480, 25, 20, 3)

data augmentation

In [5]:
import cv2
aug_X_1, aug_X_2 = [], []
sigma, mean = 1, 0
for _ in range(train_X.shape[0]):
  aug_X_1.append(cv2.GaussianBlur(train_X[_], (3, 3), 0))
  aug_X_2.append(np.random.randn(train_X[_].shape[0], train_X[_].shape[1], train_X[_].shape[2])*sigma + mean)
aug_X_1 = np.asarray(aug_X_1)
aug_X_2 = np.asarray(aug_X_2)

In [6]:
#split validation dataset
sample_len = train_X.shape[0]
splite_ratio = 0.1
valid_num = int(sample_len*splite_ratio)
valid_idx = np.random.randint(low=0, high=sample_len, size=valid_num) #生成随机数，validation的下标们
train_idx = np.asarray(list(set(list(range(sample_len))) - set(list(valid_idx)))) #set集合可以来做减法，交并集

valid_X = train_X[valid_idx, :, :, :]
valid_y = train_y[valid_idx]
train_X = train_X[train_idx, :, :, :]
train_y = train_y[train_idx]

aug_valid_X1 = aug_X_1[valid_idx, :, :, :]
aug_valid_X2 = aug_X_2[valid_idx, :, :, :]
aug_train_X1 = aug_X_1[train_idx, :, :, :]
aug_train_X2 = aug_X_2[train_idx, :, :, :]

In [7]:
## import tf and check gpu is_available
import tensorflow as tf
assert tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [8]:
## build model
batch_size = 64
views = 2
assert views == 2
temperature = 2

def build_model(input_1, input_2):
  inputs = tf.concat([input_1, input_2], axis=0)

  # CONV_1
  C_1 = tf.keras.layers.Conv2D(32, (3, 3), padding="SAME", name="C_1")(inputs)
  L_1 = tf.keras.layers.LeakyReLU()(C_1)
  D_1 = tf.keras.layers.Dropout(rate=0.5)(L_1)

  # CONV_2
  C_2 = tf.keras.layers.Conv2D(32, (3, 3), padding="SAME", name="C_2")(D_1)
  B_2 = tf.keras.layers.BatchNormalization(name="B_2")(C_2)
  L_2 = tf.keras.layers.LeakyReLU()(B_2)

  # CONV_3
  C_3_IN = tf.concat((L_2, D_1), axis=3)
  C_3 = tf.keras.layers.Conv2D(64, (3, 3), padding="SAME", name="C_3")(C_3_IN)
  L_3 = tf.keras.layers.LeakyReLU()(C_3)
  D_3 = tf.keras.layers.Dropout(rate=0.5)(L_3)

  # MAX_POOLING_1
  M_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="SAME")(D_3)

  # CONV_4
  C_4 = tf.keras.layers.Conv2D(64, (3, 3), padding="SAME", name="C_4")(M_1)
  B_4 = tf.keras.layers.BatchNormalization(name="B_4")(C_4)
  L_4 = tf.keras.layers.LeakyReLU()(B_4)
  D_4 = tf.keras.layers.Dropout(rate=0.5)(L_4)

  # CONV_5
  C_5 = tf.keras.layers.Conv2D(64, (3, 3), padding="SAME", name="C_5")(D_4)
  B_5 = tf.keras.layers.BatchNormalization(name="B_5")(C_5)
  L_5 = tf.keras.layers.LeakyReLU()(B_5)

  # CONV_6
  C_6_IN = tf.concat((L_5, D_4), axis=3)
  C_6 = tf.keras.layers.Conv2D(128, (3, 3), padding="SAME", name="C_6")(C_6_IN)
  L_6 = tf.keras.layers.LeakyReLU()(C_6)
  D_6 = tf.keras.layers.Dropout(rate=0.5)(L_6)

  # MAX_POOLING_2
  M_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="SAME")(D_6)
  print(M_2.shape)
  # FC
  F_IN = tf.reshape(M_2, [-1, 7 * 5 * 128])
  F_1 = tf.keras.layers.Dense(512, name="simCLR_F_1")(F_IN)#非线性分类器
  F_1_L = tf.keras.layers.ReLU()(F_1)
  F_2 = tf.keras.layers.Dense(256, name="simCLR_F_2")(F_1_L)
  F_2_L = tf.keras.layers.ReLU()(F_2)
  features = tf.keras.layers.Dense(128, name="simCLR_F_3")(F_2_L)

  # reconstruct logits
  labels = tf.concat([tf.range(start=0, limit=batch_size) for _ in range(views)], axis=0) #连接在一个维度的张量，range是创造一系列数字，labels现在的维度 128
  labels = tf.cast(tf.expand_dims(labels, axis=0) == tf.expand_dims(labels, axis=1), dtype=tf.float32) #更改维度，行维度变成和列维度一样，变换类型,128*128


  features = features / tf.math.reduce_euclidean_norm(features, axis=1, keepdims=True) #模为1
  similarity_matrix = tf.matmul(features, tf.transpose(features, [1, 0])) #余弦相似度

  mask = tf.linalg.eye(labels.shape[0], dtype=tf.bool)
  labels = tf.reshape(labels[~mask], [labels.shape[0], -1])
  similarity_matrix = tf.reshape(similarity_matrix[~mask], [labels.shape[0], -1])#？
  positives = tf.reshape(similarity_matrix[tf.cast(labels, dtype=tf.bool)], [labels.shape[0], -1])#？
  negatives = tf.reshape(similarity_matrix[~tf.cast(labels, dtype=tf.bool)], [similarity_matrix.shape[0], -1])#？

  logits = tf.concat([positives, negatives], axis=1)#？
  logits = logits / temperature
  # print(logits.shape[0])
  # labels = tf.zeros(logits.shape[0], dtype=tf.float32)
  return logits

input_1 = tf.keras.Input(shape=[25, 20, 3])
input_2 = tf.keras.Input(shape=[25, 20, 3])
outputs = build_model(input_1, input_2)
model = tf.keras.Model((input_1, input_2), outputs)
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(reduction='sum_over_batch_size'),  #将数字编码转化成one-hot编码格式，然后对one-hot编码格式的数据（真实标签值）与预测出的标签值使用交叉熵损失函数
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()] #检查y_true中的值（本身就是index）与y_pred中的最大值对应的index是否相等，针对稀疏情况的多分类
)

(None, 7, 5, 128)


In [9]:
def data_generator(X_t, X_t_a):
  while True:
    random_idx = np.random.randint(0, high=X_t.shape[0], size=batch_size)
    input_1 = X_t[random_idx]
    input_2 = X_t_a[random_idx]
    target = np.zeros(batch_size*views, dtype=int)#？
    yield ((input_1, input_2), target)



ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/ckpt/high_gamma_pretrain_simclr/conv6net.h5",
    mointor="val_acc",
    save_best_only=True,
    save_weights_only=True,
    mode="max"  #val_acc最大的
)

model.fit(
    x=data_generator(aug_train_X1, train_X), 
    epochs=100,  #20,30,
    steps_per_epoch=int(train_X.shape[0]//batch_size*2), #？
    verbose="auto",
    validation_data=data_generator(aug_valid_X1, valid_X),
    validation_steps=int(valid_X.shape[0]//batch_size*2), #？
    callbacks=[ckpt],
    shuffle=True 
)

Epoch 1/100
380/380 [==============================] - 51s 51ms/step - loss: 2.9222 - sparse_categorical_accuracy: 0.8352 - val_loss: 3.5427 - val_sparse_categorical_accuracy: 0.7057
Epoch 2/100
380/380 [==============================] - 17s 45ms/step - loss: 2.9996 - sparse_categorical_accuracy: 0.7359 - val_loss: 2.8430 - val_sparse_categorical_accuracy: 0.9154
Epoch 3/100
380/380 [==============================] - 17s 45ms/step - loss: 2.9813 - sparse_categorical_accuracy: 0.8313 - val_loss: 3.2496 - val_sparse_categorical_accuracy: 0.9180
Epoch 4/100
380/380 [==============================] - 17s 45ms/step - loss: 2.6414 - sparse_categorical_accuracy: 0.7758 - val_loss: 3.1520 - val_sparse_categorical_accuracy: 0.9427
Epoch 5/100
380/380 [==============================] - 17s 45ms/step - loss: 2.4977 - sparse_categorical_accuracy: 0.7321 - val_loss: 2.7927 - val_sparse_categorical_accuracy: 0.9191
Epoch 6/100
380/380 [==============================] - 17s 45ms/step - loss: 2.4695 -